# Omitir la codificación del nombre de archivo

5.14

- Problema
        
        Desea realizar operaciones de E / S de archivos utilizando nombres de archivo sin formato que no se han decodificado o codificado según la codificación de nombre de archivo predeterminada.


+ Solución
        
        De forma predeterminada, todos los nombres de archivo se codifican y decodifican de acuerdo con la codificación de texto reconvertido por sys.getfilesystemencoding (). 

Por ejemplo:

In [2]:
import sys


In [3]:
sys.getfilesystemencoding()

'utf-8'

In [4]:
# Escribir un archivo usando un nombre de archivo unicode
with open('jalape\xf1o.txt', 'w') as f:

    f.write('Spicy!')

In [5]:
# Listado de directorios (decodificado)
import os
os.listdir()

['cap5-1-7.ipynb',
 'cap5-8-13.ipynb',
 'cap5-anexo-mmap-paralelo.ipynb',
 'cap5_part_03.ipynb',
 'data',
 'data.bin',
 'ejemplo.txt',
 'ejemplo_pathlib.txt',
 'jalapeño.txt',
 'newsample.bin',
 'readme.txt',
 'saludo.txt',
 'saludo2.txt',
 'sample.bin',
 'shared_data.bin',
 'somefile',
 'somefile.bin',
 'somefile.data',
 'somefile.txt',
 'somefile2.bin',
 'somefile3',
 'somefile4']

In [11]:
# Listado de directorios (sin procesar)
os.listdir(b'.')

[b'cap5-1-7.ipynb',
 b'cap5-8-13.ipynb',
 b'cap5-anexo-mmap-paralelo.ipynb',
 b'cap5_part_03.ipynb',
 b'data',
 b'data.bin',
 b'ejemplo.txt',
 b'ejemplo_pathlib.txt',
 b'jalape\xc3\xb1o.txt',
 b'newsample.bin',
 b'readme.txt',
 b'saludo.txt',
 b'saludo2.txt',
 b'sample.bin',
 b'shared_data.bin',
 b'somefile',
 b'somefile.bin',
 b'somefile.data',
 b'somefile.txt',
 b'somefile2.bin',
 b'somefile3',
 b'somefile4']

# Abrir archivo con nombre de archivo sin procesar

In [8]:
with open(b'jalape\xc3\xb1o.txt') as f:
    print(f.read())

Spicy!


In [9]:
# Abrir archivo con nombre de archivo decodificado
with open('jalapeño.txt') as f:
    print(f.read())

Spicy!


Como puede ver en las dos últimas operaciones, el manejo del nombre de archivo cambia ligeramente cuando se proporcionan cadenas de bytes a funciones relacionadas con archivos, como open () y os.listdir ().

En circunstancias normales, no debería tener que preocuparse por la codificación y decodificación del nombre de archivo; las operaciones normales de nombre de archivo deberían funcionar. 
Sin embargo, muchos sistemas operativos pueden permitir que un usuario, por accidente o malicia, cree archivos con nombres que no se ajustan a las reglas de codificación esperadas. Dichos nombres de archivo pueden romper misteriosamente los programas de Python que funcionan con muchos archivos.

Leer directorios y trabajar con nombres de archivo como bytes sin codificar sin formato tiene el potencial de evitar tales problemas, aunque a costa de la conveniencia de la programación.

# Impresión de nombres de archivo incorrectos

5.15

- Problema
        
        Su programa recibió una lista de directorios, pero cuando intentó imprimir los nombres de archivo, se bloqueó con una excepción UnicodeEncodeError y un mensaje críptico sobre "sustitutos No permitido."


- Solución
        
        Cuando imprima nombres de archivo de origen desconocido, utilice esta convención para evitar errores:
        
los exemplos de esta receta se encuentran en el archivo Error_string.py ya que en el entorno de jupyter se genera un error-

In [22]:
# este metodo funciona pero no en un entorno jupyter ???

def bad_filename(filename):
    return repr(filename)[1:-1]

filename="b\udce4d.txt"


In [23]:
print(filename)

b�d.txt


In [24]:

print(bad_filename(filename))

b\udce4d.txt


In [31]:
def bad_filename(filename):
    import sys
    temp = filename.encode(sys.getfilesystemencoding(), errors='surrogateescape')
    return temp.decode('latin-1')

archivos = ['spam.py', 'b\udce4d.txt', 'foo.txt']

for i in archivos:
    print(bad_filename(i))

spam.py
bäd.txt
foo.txt


Es probable que la mayoría de los lectores ignoren esta receta. Sin embargo, si está escribiendo scripts de misión crítica que necesitan funcionar de manera confiable con los nombres de archivo y el sistema de archivos, es algo para pensarlo.   
De lo contrario, es posible que lo llamen nuevamente a la oficina durante el fin de semana para depurar un error aparentemente inescrutable.

In [28]:
"\udce0".encode("utf-8",errors='surrogateescape').decode("cp1252")

'à'

In [29]:
"í".encode("utf-8",errors='surrogateescape')

b'\xc3\xad'

In [30]:
'\xc3\xad'.encode("cp1252",errors='surrogateescape').decode("utf-8")

'í'

# Agregar o cambiar la codificación de un archivo ya abierto

5.16

- Problema
       
        Desea agregar o cambiar la codificación Unicode de un archivo ya abierto sin cerrarlo primero.


- Solución
       
        Si desea agregar codificación / decodificación Unicode a un objeto de archivo ya existente que es abierto en modo binario, envuélvalo con un objeto io.TextIOWrapper ().   
        
Por ejemplo:

In [3]:
import urllib.request
import io

In [4]:
u = urllib.request.urlopen('http://www.python.org')
f = io.TextIOWrapper(u , encoding='utf-8')
text = f.read()

Si desea cambiar la codificación de un archivo en modo texto ya abierto, use su detach () método para eliminar la capa de codificación de texto existente antes de reemplazarla por una nueva.
A continuación, se muestra un ejemplo de cómo cambiar la codificación en sys.stdout:

Hacer esto podría romper la salida de su terminal. Solo tiene la intención de ilustrar.
```python
>>> import sys
>>> sys.stdout.encoding
'UTF-8'
>>> sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='latin-1')
>>> sys.stdout.encoding
'latin-1'
>>>
```

El sistema de E / S está construido como una serie de capas. Puedes ver las capas tú mismo probando esto
ejemplo simple que involucra un archivo de texto:

In [6]:
f = open('sample.txt','w')
f

<_io.TextIOWrapper name='sample.txt' mode='w' encoding='UTF-8'>

In [7]:
f.buffer

<_io.BufferedWriter name='sample.txt'>

In [8]:
f.buffer.raw

<_io.FileIO name='sample.txt' mode='wb' closefd=True>

En este ejemplo, io.TextIOWrapper es una capa de manejo de texto que codifica y decodifica Unicode, io.BufferedWriter es una capa de E / S almacenada en búfer que maneja datos binarios y io.FileIO es un archivo sin formato que representa el descriptor de archivo de bajo nivel en el sistema operativo.
Agregar o cambiar la codificación de texto implica agregar o cambiar la parte superior io.TextIOWrapper layer.
Como regla general, no es seguro manipular directamente las diferentes capas accediendo al atributos mostrados.   
Por ejemplo, vea lo que sucede si intenta cambiar la codificación usando
esta tecnica:

In [9]:
f = io.TextIOWrapper(f.buffer, encoding='latin-1')
f

<_io.TextIOWrapper name='sample.txt' encoding='latin-1'>

In [10]:
f.write('Hello')

5

In [11]:
f.close()

In [12]:
f = open('sample2.txt', 'w')
f

<_io.TextIOWrapper name='sample2.txt' mode='w' encoding='UTF-8'>

In [13]:
b = f.detach()
b

<_io.BufferedWriter name='sample2.txt'>

In [15]:
b.write(b"hola")

4

In [16]:
f = io.TextIOWrapper(b, encoding='latin-1')
f

<_io.TextIOWrapper name='sample2.txt' encoding='latin-1'>

In [17]:
f.close()

Aunque se ha mostrado cambiar la codificación, también es posible utilizar esta técnica para cambiar el manejo de líneas, la política de errores y otros aspectos del manejo de archivos.   
Por ejemplo: 

```python
>>> sys.stdout = io.TextIOWrapper(sys.stdout.detach(), 
                                  encoding = 'ascii', 
                                  errors   = 'xmlcharrefreplace')
>>> print('Jalape\u00f1o')
'Jalape&#241;o'
>>>
```
Observe cómo el carácter no ASCII ñ ha sido reemplazado por &#241; en la salida

# Anexo Moderno: Manejo de Nombres de Archivo con Python 3.6+


## 1. Manejo Moderno de Nombres de Archivo con pathlib

### Ventajas de pathlib:
- API orientada a objetos más intuitiva
- Manejo automático de separadores de ruta (`/` vs `\`)
- Métodos convenientes para operaciones comunes
- Mejor manejo de codificación de nombres de archivo

In [1]:
from pathlib import Path
import sys

# Crear archivo con caracteres Unicode usando pathlib
archivo_unicode = Path('jalapeño_moderno.txt')

# Escribir contenido
archivo_unicode.write_text('¡Picante! 🌶️', encoding='utf-8')

# Leer contenido
contenido = archivo_unicode.read_text(encoding='utf-8')
print(f"Contenido: {contenido}")

# Verificar existencia
print(f"¿Existe? {archivo_unicode.exists()}")

# Obtener información
print(f"Nombre: {archivo_unicode.name}")
print(f"Sufijo: {archivo_unicode.suffix}")
print(f"Ruta absoluta: {archivo_unicode.absolute()}")

Contenido: ¡Picante! 🌶️
¿Existe? True
Nombre: jalapeño_moderno.txt
Sufijo: .txt
Ruta absoluta: d:\recetas-python\Capitulo 5 (Archivos u I\jalapeño_moderno.txt


## 2. Listar Directorio de forma moderna

In [2]:
from pathlib import Path

# Listar todos los archivos en el directorio actual
directorio = Path('.')

print("=== Todos los archivos ===")
for archivo in directorio.iterdir():
    tipo = "📁" if archivo.is_dir() else "📄"
    print(f"{tipo} {archivo.name}")

print("\n=== Solo archivos .txt ===")
for txt in directorio.glob('*.txt'):
    print(f"📄 {txt.name} ({txt.stat().st_size} bytes)")

print("\n=== Búsqueda recursiva de .py ===")
for py in directorio.rglob('*.py'):
    print(f"🐍 {py.relative_to(directorio)}")

=== Todos los archivos ===
📄 ACTUALIZACIONES_MODERNAS.md
📄 cap5-1-7.ipynb
📄 cap5-14-16..ipynb
📄 cap5-8-13.ipynb
📄 cap5-anexo-mmap-paralelo.ipynb
📄 cap5_part_03_anexo_moderno.ipynb
📄 data
📄 data.bin
📄 demo_manejo_moderno.py
📄 ejemplo.txt
📄 ejemplo_pathlib.txt
📄 jalapeño.txt
📄 jalapeño_moderno.txt
📄 newsample.bin
📄 readme.txt
📄 saludo.txt
📄 saludo2.txt
📄 sample.bin
📄 shared_data.bin
📄 somefile
📄 somefile.bin
📄 somefile.data
📄 somefile.txt
📄 somefile2.bin
📄 somefile3
📄 somefile4

=== Solo archivos .txt ===
📄 ejemplo.txt (442 bytes)
📄 ejemplo_pathlib.txt (20 bytes)
📄 jalapeño.txt (6 bytes)
📄 jalapeño_moderno.txt (18 bytes)
📄 readme.txt (91 bytes)
📄 saludo.txt (65 bytes)
📄 saludo2.txt (26 bytes)
📄 somefile.txt (190 bytes)

=== Búsqueda recursiva de .py ===
🐍 demo_manejo_moderno.py
